In [42]:
from peptdeep.model.ms2 import pDeepModel, normalize_fragment_intensities
from peptdeep.model.rt import IRT_PEPTIDE_DF
from alphabase.spectral_library.flat import SpecLibFlat
import numpy as np

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Understanding the New Pretrained Weights Format  

This notebook explores various use cases and the reasons behind adopting the new pretrained weights format.  

## Summary  

The new format introduces two key improvements:  

1. **Fixes incorrect behavior** in certain valid use cases when using the legacy weight format.  
2. **Enhances flexibility, adaptability, and transparency** when using the pretrained MS2 models in **PeptDeep**.  

## Key Considerations  

- The new weights format **was not obtained from retraining** of the models.  
- So it **does not provide performance improvements** over the legacy weights but significantly expands the range of supported use cases.  

## Use Cases  

We categorize use cases based on two main tasks:  

1. **Prediction**  
2. **Transfer Learning**  

For each task, we will:  
- Show that legacy weights are **still** supported by the current implementation (Without the new use cases). 
- Highlight the **limitations** of the legacy weights format.  
- Present **new use cases** that are now supported with the updated format.  


Supported use cases with the new format:
| Fragtypes use case              , Override from weights (*) | Safe to predict | Safe to train |
|-----------------------------------------------------------|----------------|--------------|
| requested = supported   (1)       ,         False                 |      ✅          |      ✅        |
| requested ⊆ supported    (2)     ,            False              |         ✅      |       ❌       |
| requested ⊈ supported     (3)    ,             False             |        ❌        |     ❌         |
|                  Any               ,            True              |       ✅         |      ✅        |

(1) The ideal use case (only one supported by the old implementation) where users know and request exactly the same farg types supported in the model weights.

(2) Users only need to predict a subset of the frag types supported by the loaded weights. 

(3) Users requested charged frag types that are not supported (can easily be identified now since we don't only look at the number of requested frag types).

(*) `Override from weights` is the new argument added to the MS2 model, this allow users to load models without knowing exactly what are the supported frag types in a pretrained model. So this overrides the requested frag types and uses all supported frag types by the loaded model.

Safe to train: Any model is automatically set to 'safe to train' when the `train` function is called-- by modifying *only* the underlying model output head to align with the requested frag types. After training, the model will automatically be in a 'safe to predict' state. 


In [43]:
def get_prediction_dataset():
    df=IRT_PEPTIDE_DF.copy()
    df['charge'] = 2
    df['mods'] = ''
    df['mod_sites'] = ''
    # sort by nAA
    df = df.sort_values('nAA')
    idxes = np.zeros(len(df)+1,dtype=np.int64)
    idxes[1:] = np.cumsum(df.nAA.values-1)
    df['frag_start_idx'] = idxes[:-1]
    df['frag_stop_idx'] = idxes[1:]
    df['nce'] = 30
    df['instrument'] = "Lumos"
    # sort by 
    return df
get_prediction_dataset().head()

,sequence,pep_name,irt,mods,mod_sites,nAA,charge,frag_start_idx,frag_stop_idx,nce,instrument
0,LGGNEQVTR,RT-pep a,-24.92,,,9,2,0,8,30,Lumos
3,YILAGVENSK,RT-pep d,19.79,,,10,2,8,17,30,Lumos
4,TPVISGGPYEYR,RT-pep e,28.71,,,12,2,17,28,30,Lumos
5,TPVITGAPYEYR,RT-pep f,33.38,,,12,2,28,39,30,Lumos
8,GTFIIDPGGVIR,RT-pep i,70.52,,,12,2,39,50,30,Lumos


#### What is actually **new** about the "new" weights ? 

- Both formats share the same exact underlying weights for the model, the only difference is with the new format we save the charged frag types used during training in the weights file. Which will allow supporting the new use cases.
- Both models support the following fragament types:
'b_z1', 'b_z2', 'y_z1', 'y_z2', 'b_modloss_z1', 'b_modloss_z2', 'y_modloss_z1', 'y_modloss_z2'

- To see what fragment types are supported in the alpha ecosystem check alphabase.fragment.FRAGMENT_TYPES

In [ ]:
legacy_path = "../legacy_pretrained_models/generic/ms2.pth"
new_path = "../new_pretrained_models/generic/ms2.pth"

# 1. Ms2 Prediction 

## 1.1 Starting with the Legacy weights format
To show what the new format solve, lets consider three different use cases of loading a pretrained model.

a) Using incorrect *len* of frag types when initialization (Will raise mismatch error)
- Which means users can not predict a subset of the fragment types supported by the loaded pretrained model

In [45]:
try: 
    model_interface = pDeepModel(charged_frag_types=['b_z1', 'b_z2', 'b_modloss_z1', 'b_modloss_z2'])
    model_interface.load(legacy_path)
except Exception as e:
    print(e)

Error(s) in loading state_dict for ModelMS2Bert:
	size mismatch for output_nn.nn.2.weight: copying a param with shape torch.Size([4, 64]) from checkpoint, the shape in current model is torch.Size([2, 64]).
	size mismatch for output_nn.nn.2.bias: copying a param with shape torch.Size([4]) from checkpoint, the shape in current model is torch.Size([2]).
	size mismatch for modloss_nn.1.nn.2.weight: copying a param with shape torch.Size([4, 64]) from checkpoint, the shape in current model is torch.Size([2, 64]).
	size mismatch for modloss_nn.1.nn.2.bias: copying a param with shape torch.Size([4]) from checkpoint, the shape in current model is torch.Size([2]).


b) Using the correct *len* of frag types when initialization ***EVEN*** if the requested fragment types are completely different.(**Worst** use case)   
- It's important to notice that the old format **won't** raise an error and the prediction goes through (even with misleading column names)

In [46]:
# Notice replacing the y_z1 and y_z2 with x_z1 and x_z2 and the model is loaded successfully and we get an incorrect prediction
model_interface = pDeepModel(charged_frag_types=['b_z1', 'b_z2', 'x_z1', 'x_z2', 'b_modloss_z1', 'b_modloss_z2', 'y_modloss_z1', 'y_modloss_z2'])
model_interface.load(legacy_path)

In [47]:
preds = model_interface.predict(get_prediction_dataset())
preds.head()

,b_z1,b_z2,x_z1,x_z2,b_modloss_z1,b_modloss_z2,y_modloss_z1,y_modloss_z2
0,0.000000,0.0,1.000000,0.004739,0.0,0.0,0.0,0.0
1,0.162034,0.0,0.360414,0.000000,0.0,0.0,0.0,0.0
2,0.046660,0.0,0.109920,0.005516,0.0,0.0,0.0,0.0
3,0.018628,0.0,0.203326,0.000000,0.0,0.0,0.0,0.0
4,0.013530,0.0,0.267507,0.000000,0.0,0.0,0.0,0.0


c) Using exactly the same charged frags (and same order) as used during training. (**Ideal** use case and the only one supported by the legacy format)

In [48]:
# Ideal use case requested frag types == training frag types
model_interface = pDeepModel(charged_frag_types=['b_z1', 'b_z2', 'y_z1', 'y_z2', 'b_modloss_z1', 'b_modloss_z2', 'y_modloss_z1', 'y_modloss_z2'])
model_interface.load(legacy_path)


In [49]:
legacy_full_preds = model_interface.predict(get_prediction_dataset())
legacy_full_preds.head()

,b_z1,b_z2,y_z1,y_z2,b_modloss_z1,b_modloss_z2,y_modloss_z1,y_modloss_z2
0,0.000000,0.0,1.000000,0.004739,0.0,0.0,0.0,0.0
1,0.162034,0.0,0.360414,0.000000,0.0,0.0,0.0,0.0
2,0.046660,0.0,0.109920,0.005516,0.0,0.0,0.0,0.0
3,0.018628,0.0,0.203326,0.000000,0.0,0.0,0.0,0.0
4,0.013530,0.0,0.267507,0.000000,0.0,0.0,0.0,0.0


In [50]:
"""
If you don't have the new weights format uncomment the following line and run the cell 
after loading the legacy model weights with the correct frag types (last 2 cells)
it should  save the new weights in the new_path.
"""
# model_interface.save(new_path)

"\nIf you don't have the new weights format uncomment the following line and run the cell \nafter loading the legacy model weights with the correct frag types (last 2 cells)\nit should  save the new weights in the new_path.\n"

## 1.2 Benefits of the new parameter format shown with different use-cases.

a) Using exactly the same charged frags (and same order) as used during training. (**Ideal** use case)

In [51]:
model_interface = pDeepModel(charged_frag_types=['b_z1', 'b_z2', 'y_z1', 'y_z2', 'b_modloss_z1', 'b_modloss_z2', 'y_modloss_z1', 'y_modloss_z2'])
model_interface.load(new_path)

In [52]:
new_full_preds = model_interface.predict(get_prediction_dataset())
# verify the predictions are the same
assert np.allclose(legacy_full_preds.values, new_full_preds.values, atol=1e-5)
new_full_preds.head()

,b_z1,b_z2,y_z1,y_z2,b_modloss_z1,b_modloss_z2,y_modloss_z1,y_modloss_z2
0,0.000000,0.0,1.000000,0.004739,0.0,0.0,0.0,0.0
1,0.162034,0.0,0.360414,0.000000,0.0,0.0,0.0,0.0
2,0.046660,0.0,0.109920,0.005516,0.0,0.0,0.0,0.0
3,0.018628,0.0,0.203326,0.000000,0.0,0.0,0.0,0.0
4,0.013530,0.0,0.267507,0.000000,0.0,0.0,0.0,0.0


b) Using incorrect *len* of frag types during initialization but still a valid subset of what was used during training. 

Examples:
- 1) Excluding the modloss frags, preivously done by setting mask_modloss = True

In [53]:
# Excluding the modloss fragment types
model_interface = pDeepModel(charged_frag_types=['b_z1', 'b_z2', 'y_z1', 'y_z2'])
model_interface.load(new_path)
print(f"Model Interface has charged_frag_types {model_interface.charged_frag_types}")
print(f"Supported charged_frag_types in the loaded weights  {model_interface.model.supported_charged_frag_types}")

Model Interface has charged_frag_types ['b_z1', 'b_z2', 'y_z1', 'y_z2']
Supported charged_frag_types in the loaded weights  ['b_z1', 'b_z2', 'y_z1', 'y_z2', 'b_modloss_z1', 'b_modloss_z2', 'y_modloss_z1', 'y_modloss_z2']


In [54]:
# verify that the predictions are the same with the legacy model for the selected subset of charged_frag_types
new_subset_preds = model_interface.predict(get_prediction_dataset())
assert np.allclose(legacy_full_preds[new_subset_preds.columns], new_subset_preds)
new_subset_preds.head()

,b_z1,b_z2,y_z1,y_z2
0,0.000000,0.0,1.000000,0.004739
1,0.162034,0.0,0.360414,0.000000
2,0.046660,0.0,0.109920,0.005516
3,0.018628,0.0,0.203326,0.000000
4,0.013530,0.0,0.267507,0.000000


- 2) Excluding frag types that are not modloss (*New* use case)

In [ ]:
# Excluding the y fragments while keeping the modloss fragments
model_interface = pDeepModel(charged_frag_types=['b_z1', 'b_z2', 'b_modloss_z1', 'b_modloss_z2'])
model_interface.load(new_path)

In [56]:
# verify that the predictions are the same with the legacy model for the selected subset of charged_frag_types
new_subset_preds = model_interface.predict(get_prediction_dataset())
assert np.allclose(legacy_full_preds[new_subset_preds.columns], new_subset_preds)
new_subset_preds.head()

,b_z1,b_z2,b_modloss_z1,b_modloss_z2
0,0.000000,0.0,0.0,0.0
1,0.162034,0.0,0.0,0.0
2,0.046660,0.0,0.0,0.0
3,0.018628,0.0,0.0,0.0
4,0.013530,0.0,0.0,0.0


c) Requesting fragment types that are not supported by the loaded pretrained model. 

Using the new format; the supported charged frag types are explicit and transparent, so when a user request frag types that are not supported we can detect and raise an interpretable *error* (**New** feature)

In [57]:
try:
    model_interface = pDeepModel(charged_frag_types=['x_z1', 'x_z2'])
    model_interface.load(new_path)

    new_subset_preds = model_interface.predict(get_prediction_dataset())
except Exception as e:
    print(e)


The model is not safe to use for prediction. This might mean that the requested charged_frag_types ['x_z1', 'x_z2'] are not a subset of the charged_frag_types used to train the loaded pretrained model ['b_z1', 'b_z2', 'y_z1', 'y_z2', 'b_modloss_z1', 'b_modloss_z2', 'y_modloss_z1', 'y_modloss_z2']. Please choose a subset of the supported charged_frag_types or retrain the model with the requested charged_frag_types.


d) **Even** if the user requested correct *len* of frag types but the requested frag types are not a subset of what was used during training we should raise an *error* (Solving what was previously the **worst** case)

In [58]:
try:
    model_interface =  pDeepModel(charged_frag_types=['c_z1', 'c_z2', 'y_z1', 'y_z2', 'x_z1', 'x_z2', 'y_modloss_z1', 'y_modloss_z2'])
    model_interface.load(new_path)

    new_subset_preds = model_interface.predict(get_prediction_dataset())
except Exception as e:
    print(e)


The model is not safe to use for prediction. This might mean that the requested charged_frag_types ['c_z1', 'c_z2', 'x_z1', 'x_z2', 'y_z1', 'y_z2', 'y_modloss_z1', 'y_modloss_z2'] are not a subset of the charged_frag_types used to train the loaded pretrained model ['b_z1', 'b_z2', 'y_z1', 'y_z2', 'b_modloss_z1', 'b_modloss_z2', 'y_modloss_z1', 'y_modloss_z2']. Please choose a subset of the supported charged_frag_types or retrain the model with the requested charged_frag_types.


e) User has a weights file and want to predict all fragment types supported **without** knowing what exactly was used during training (**New** use case)

Notice how the requested frag types are overridden in the model interface

In [59]:
model_interface =  pDeepModel(
    charged_frag_types=['c_z1'], # Will be overridden by the supported charged_frag_types in the loaded weights
    override_from_weights=True
    )
model_interface.load(new_path)

print(f"Model Interface has requested charged_frag_types {model_interface.charged_frag_types}")
print(f"Supported charged_frag_types in the loaded weights  {model_interface.model.supported_charged_frag_types}")

Model Interface has requested charged_frag_types ['b_z1', 'b_z2', 'y_z1', 'y_z2', 'b_modloss_z1', 'b_modloss_z2', 'y_modloss_z1', 'y_modloss_z2']
Supported charged_frag_types in the loaded weights  ['b_z1', 'b_z2', 'y_z1', 'y_z2', 'b_modloss_z1', 'b_modloss_z2', 'y_modloss_z1', 'y_modloss_z2']


In [60]:
new_full_preds = model_interface.predict(get_prediction_dataset())
# verify the predictions are the same
assert np.allclose(legacy_full_preds.values, new_full_preds.values, atol=1e-5)
new_full_preds.head()

,b_z1,b_z2,y_z1,y_z2,b_modloss_z1,b_modloss_z2,y_modloss_z1,y_modloss_z2
0,0.000000,0.0,1.000000,0.004739,0.0,0.0,0.0,0.0
1,0.162034,0.0,0.360414,0.000000,0.0,0.0,0.0,0.0
2,0.046660,0.0,0.109920,0.005516,0.0,0.0,0.0,0.0
3,0.018628,0.0,0.203326,0.000000,0.0,0.0,0.0,0.0
4,0.013530,0.0,0.267507,0.000000,0.0,0.0,0.0,0.0


f) Even If users requested the correct frag types but with an *incorrect* order, they are implicitly ordered using `alphabase.sort_charged_frag_types`

In [61]:
model_interface = pDeepModel(charged_frag_types=['y_z2', 'y_z1', 'b_z2', 'b_z1', 'b_modloss_z1', 'b_modloss_z2', 'y_modloss_z1', 'y_modloss_z2'])
model_interface.load(new_path)
print(f"Model Interface has requested charged_frag_types {model_interface.charged_frag_types}")
print(f"Supported charged_frag_types in the loaded weights  {model_interface.model.supported_charged_frag_types}")


Model Interface has requested charged_frag_types ['b_z1', 'b_z2', 'y_z1', 'y_z2', 'b_modloss_z1', 'b_modloss_z2', 'y_modloss_z1', 'y_modloss_z2']
Supported charged_frag_types in the loaded weights  ['b_z1', 'b_z2', 'y_z1', 'y_z2', 'b_modloss_z1', 'b_modloss_z2', 'y_modloss_z1', 'y_modloss_z2']


# 2. Ms2 transfer learning

In [ ]:
trainin_data_path = "../data/2oh_evidence_txt_0_batch_0.hdf"
speclib = SpecLibFlat()
speclib.load_hdf(trainin_data_path)
speclib.fragment_intensity_df["b_modloss_z1"] = 0
speclib.fragment_intensity_df["b_modloss_z2"] = 0
speclib.fragment_intensity_df["y_modloss_z1"] = 0
speclib.fragment_intensity_df["y_modloss_z2"] = 0
frgament_types_in_data = speclib.fragment_intensity_df.columns

speclib.precursor_df['nce'] = 30
speclib.precursor_df['instrument'] = "Lumos"
# sample only 100 samples
speclib.precursor_df = speclib.precursor_df.sample(100)

# normalize intensity 
normalize_fragment_intensities(speclib.precursor_df, speclib.fragment_intensity_df)
print(f"Fragment types in the training data: {frgament_types_in_data}")

Fragment types in the training data: Index(['a_z1', 'a_z2', 'b_z1', 'b_z2', 'c_z1', 'c_z2', 'x_z1', 'x_z2', 'y_z1',
       'y_z2', 'z_z1', 'z_z2', 'b_H2O_z1', 'b_H2O_z2', 'b_NH3_z1', 'b_NH3_z2',
       'c_lossH_z1', 'c_lossH_z2', 'y_H2O_z1', 'y_H2O_z2', 'y_NH3_z1',
       'y_NH3_z2', 'z_addH_z1', 'z_addH_z2', 'b_modloss_z1', 'b_modloss_z2',
       'y_modloss_z1', 'y_modloss_z2'],
      dtype='object')


## 2.1 User loading a legacy model 

Using incorrect *len* of frag types when initialization (Will raise a mismatch error)


In [70]:
try:
    target_frag_types = ['b_z1', 'b_z2', 'b_modloss_z1', 'b_modloss_z2']
    model_interface = pDeepModel(charged_frag_types=target_frag_types)
    model_interface.load(legacy_path)
    model_interface.train(precursor_df=speclib.precursor_df, fragment_intensity_df=speclib.fragment_intensity_df.loc[:,target_frag_types], epoch=1, verbose=1)
except Exception as e:
    print(e)

Error(s) in loading state_dict for ModelMS2Bert:
	size mismatch for output_nn.nn.2.weight: copying a param with shape torch.Size([4, 64]) from checkpoint, the shape in current model is torch.Size([2, 64]).
	size mismatch for output_nn.nn.2.bias: copying a param with shape torch.Size([4]) from checkpoint, the shape in current model is torch.Size([2]).
	size mismatch for modloss_nn.1.nn.2.weight: copying a param with shape torch.Size([4, 64]) from checkpoint, the shape in current model is torch.Size([2, 64]).
	size mismatch for modloss_nn.1.nn.2.bias: copying a param with shape torch.Size([4]) from checkpoint, the shape in current model is torch.Size([2]).


a) Using correct *len* of frag types when initialization (**Ideal** use case)


In [64]:
target_frag_types = ['b_z1', 'b_z2', 'y_z1', 'y_z2', 'b_modloss_z1', 'b_modloss_z2', 'y_modloss_z1', 'y_modloss_z2']
model_interface = pDeepModel(charged_frag_types=target_frag_types)
model_interface.load(legacy_path)
model_interface.train(precursor_df=speclib.precursor_df, fragment_intensity_df=speclib.fragment_intensity_df.loc[:,target_frag_types], epoch=3, verbose=1)

2025-02-27 18:19:19> Training with fixed sequence length: 0
[Training] Epoch=1, Mean Loss=0.01842992820052637
[Training] Epoch=2, Mean Loss=0.017078618622488446
[Training] Epoch=3, Mean Loss=0.01577050112084382


## 2.2 Benefits of the new parameter format for transfer learning.

Training on new fragment types that were not part of the training of the original weights (**New** feature). 
- Since the architecture to support the requested fragment types will be different lets try the most obvious solution first -- training from scratch.  


In [ ]:
# Notice we are not loading any weights but training the model from scratch
target_frag_types = ['a_z1', 'a_z2', 'b_H2O_z1', 'b_H2O_z2'] 
model_interface = pDeepModel(charged_frag_types=target_frag_types)
print("Training the model with the requested fragment types from scratch")
model_interface.train(precursor_df=speclib.precursor_df, fragment_intensity_df=speclib.fragment_intensity_df.loc[:,target_frag_types], epoch=10, verbose=1)


Training the model with the requested fragment types from scratch
2025-02-27 18:19:23> Training with fixed sequence length: 0
[Training] Epoch=1, Mean Loss=0.20166994051800835
[Training] Epoch=2, Mean Loss=0.15216446336772707
[Training] Epoch=3, Mean Loss=0.12939151955975425
[Training] Epoch=4, Mean Loss=0.1190311929417981
[Training] Epoch=5, Mean Loss=0.10276750516560343
[Training] Epoch=6, Mean Loss=0.0983661309712463
[Training] Epoch=7, Mean Loss=0.09396917579902543
[Training] Epoch=8, Mean Loss=0.08694049964348476
[Training] Epoch=9, Mean Loss=0.08316284211145507
[Training] Epoch=10, Mean Loss=0.08015417887104882


- Using transfer learning by loading the pre-trained backbone and only the prediction heads are initialized from scratch which results in a much faster convergence and reduce the risk of overfiting.
- Notice 2 things:
    - The final training loss compared to training from scratch for the same number of epochs.
    - When the requested frag types are not a subset of the supported, the model is not safe to use for prediction. After training, the model's state is automatically changed to _safe_to_predict.

In [71]:
target_frag_types = ['a_z1', 'a_z2', 'b_H2O_z1', 'b_H2O_z2'] 
model_interface = pDeepModel(charged_frag_types=target_frag_types)
model_interface.load(new_path)
print("Trying to predict when the requested fragment types are not supported by the pretrained model")
print(f"Requested fragment types: {target_frag_types}")
print(f"Supported fragment types in the loaded weights: {model_interface.model.supported_charged_frag_types}")
try: 
    # try to predict with the new model
    model_interface.predict(get_prediction_dataset())
except Exception as e:
    print(f"Error: {e}")



Trying to predict when the requested fragment types are not supported by the pretrained model
Requested fragment types: ['a_z1', 'a_z2', 'b_H2O_z1', 'b_H2O_z2']
Supported fragment types in the loaded weights: ['b_z1', 'b_z2', 'y_z1', 'y_z2', 'b_modloss_z1', 'b_modloss_z2', 'y_modloss_z1', 'y_modloss_z2']
Error: The model is not safe to use for prediction. This might mean that the requested charged_frag_types ['a_z1', 'a_z2', 'b_H2O_z1', 'b_H2O_z2'] are not a subset of the charged_frag_types used to train the loaded pretrained model ['b_z1', 'b_z2', 'y_z1', 'y_z2', 'b_modloss_z1', 'b_modloss_z2', 'y_modloss_z1', 'y_modloss_z2']. Please choose a subset of the supported charged_frag_types or retrain the model with the requested charged_frag_types.


In [72]:
print("Training the model with the requested fragment types")

model_interface.train(precursor_df=speclib.precursor_df, fragment_intensity_df=speclib.fragment_intensity_df.loc[:,target_frag_types], epoch=10, verbose=1)


Training the model with the requested fragment types
2025-02-27 18:30:05> Training with fixed sequence length: 0
[Training] Epoch=1, Mean Loss=0.08342412755721146
[Training] Epoch=2, Mean Loss=0.07767527177929878
[Training] Epoch=3, Mean Loss=0.07237229889465703
[Training] Epoch=4, Mean Loss=0.06727897334429953
[Training] Epoch=5, Mean Loss=0.06287239574723774
[Training] Epoch=6, Mean Loss=0.05764101083493895
[Training] Epoch=7, Mean Loss=0.05389009426451392
[Training] Epoch=8, Mean Loss=0.05020008898443646
[Training] Epoch=9, Mean Loss=0.04623174532834026
[Training] Epoch=10, Mean Loss=0.041998228368659817


In [73]:
print("Trying to predict after training with the requested fragment types")
try: 
    # try to predict with the new model
    preds = model_interface.predict(get_prediction_dataset())
except Exception as e:
    print(f"Error: {e}")

preds.head()

Trying to predict after training with the requested fragment types


,a_z1,a_z2,b_H2O_z1,b_H2O_z2
0,0.000000,0.0,1.000000,0.0
1,0.116859,0.0,0.316274,0.0
2,0.011386,0.0,0.044198,0.0
3,0.000000,0.0,0.092805,0.0
4,0.000000,0.0,0.125801,0.0


After training the new underlying model has requested frag types as the supported frag types. Which can then be saved and used for future prediction.

In [74]:
print(f"Model Interface has requested charged_frag_types {model_interface.charged_frag_types}")
print(f"Supported charged_frag_types in the loaded weights  {model_interface.model.supported_charged_frag_types}")

Model Interface has requested charged_frag_types ['a_z1', 'a_z2', 'b_H2O_z1', 'b_H2O_z2']
Supported charged_frag_types in the loaded weights  ['a_z1', 'a_z2', 'b_H2O_z1', 'b_H2O_z2']
